# Data Organization

## Prepare protein embedding

Get the embedding.pt file for each species's genes, each species only needs to be done once."

In [ ]:
pep_file = 'data/pep/sp1.fa' # Input protein sequences, ensure each gene has only one
tmp_dir = 'data/pep/sp1_pep_tmp' # Temporary directory

In [ ]:
!conda run -n SATURN python extract.py esm2_t48_15B_UR50D  $pep_file $tmp_dir --repr_layers 0 47 48 --include mean per_tok &> run.log &

In [ ]:
!conda run -n SATURN python merge_embedding.py $tmp_dir input sp1

# output file
# - sp1_protein_id_to_embedding.pt
# - sp1_pt.pkl

In [ ]:
pep_file = 'data/pep/sp2.fa' # Input protein sequences, ensure each gene has only one
tmp_dir = 'data/pep/sp2_pep_tmp' # Temporary directory

In [ ]:
!conda run -n SATURN python extract.py esm2_t48_15B_UR50D  $pep_file $tmp_dir --repr_layers 0 47 48 --include mean per_tok &> run.log &

In [ ]:
!conda run -n SATURN python merge_embedding.py $tmp_dir input sp2

# output file
# - sp1_protein_id_to_embedding.pt
# - sp1_pt.pkl

# Save final result

In [ ]:
# Make the csv
import pandas as pd

df = pd.DataFrame(columns=["path", "species", "embedding_path"])
df["species"] = ["sp1", "sp2"]
df["path"] = [
"data/h5ad/sp1.h5ad",
"data/h5ad/sp2.h5ad",
    ]

##############################
df["embedding_path"] = [
"data/protein_id_to_embedding/sp1_protein_id_to_embedding.pt",
"data/protein_id_to_embedding/sp2_protein_id_to_embedding.pt",
]
df.to_csv("data/saturn_data.csv", index=False)
df

In [57]:
import os
full_path = os.path.abspath("data/saturn_data.csv")

In [ ]:
!cd ~/software/SATURN && conda run -n SATURN python3 ./train-saturn.py \
    --in_data={full_path} --in_label_col="cell_type" \
        --ref_label_col="cell_type" \
    --embedding_model="ESM2" --batch_size=10240 --epochs=200 

In [61]:
# Move analysis results to current directory
!mv ~/software/SATURN/out/saturn_results/ .